In [ ]:
conda install -c conda-forge opencv

In [ ]:
# import openCv
import cv2
# import uuid
import uuid
# import operating system
import os
# import time
import time
# ignore warnings
from warnings import filterwarnings
filterwarnings("ignore")

# 2. Define images to collect

In [ ]:
labels = ['thumbsup', 'thumbsdown', 'thankyou', 'livelong']
labels = ['thankyou']
number_img = 5

In [9]:
print(number_img)

5


# 3. Setup Folders

In [11]:
Images_paths = os.path.join('Tensorflow', 'workspace', 'images', 'collectedimages')

In [36]:
print(Images_paths)

Tensorflow/workspace/images/collectedimages


In [13]:
if not os.path.exists(Images_paths):
    if os.name =='posix':
        os.makedirs(Images_paths, exist_ok=True)
    if os.name == 'nt':
        os.makedirs(Images_paths, exist_ok=True)
for label in labels:
    path = os.path.join(Images_paths, label)
    if not os.path.exists(path):
        os.makedirs (path, exist_ok=True)

# 4. Capture Images

In [15]:
for label in labels:
    cap = cv2.VideoCapture(0) # connects to the webcap or capture device
    print('Collecting images for {}'.format(label))
    time.sleep(5)
    for imgnum in range(number_img):
        print('collecting images {}'.format(imgnum))
        ret, frame = cap.read()
        imgname = os.path.join(Images_paths, label, label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname,frame)
        cv2.imshow('frame', frame)
        time.sleep(2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

[ WARN:0@1285.065] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


collecting images 0
collecting images 1
collecting images 2
collecting images 3
collecting images 4


# image labeling

In [ ]:
conda list | grep pyqt

In [ ]:
LABELING_PATH = os.path.join('Tensorflow', 'labelimg')

In [ ]:
if not os.path.exists(LABELING_PATH):
    os.makedirs(LABELING_PATH)
    !git clone https://github.com/tzutalin/labelimg {LABELING_PATH}

In [ ]:
if os.name == "posix":
    !cd {LABELING_PATH} && make qt5py3
if os.name =="nt":
    !cd {LABELING_PATH} && pyrcc5 -o libs/resources.py resources.qrc

In [ ]:
# opening the label image package
!cd {LABELING_PATH} && python labelimg.py

Cancel creation.
Image:/Users/briankimanzi/Documents/programmingLanguages/PythonProgramming/JupyterAI-Clean/Tensorflow/workspace/images/collectedimages/thankyou/thankyou.3ccfdfda-87bf-11f0-9565-1094bbd0ecec.jpg -> Annotation:/Users/briankimanzi/Documents/programmingLanguages/PythonProgramming/JupyterAI-Clean/Tensorflow/workspace/images/collectedimages/thankyou/thankyou.3ccfdfda-87bf-11f0-9565-1094bbd0ecec.xml
Image:/Users/briankimanzi/Documents/programmingLanguages/PythonProgramming/JupyterAI-Clean/Tensorflow/workspace/images/collectedimages/thankyou/thankyou.3e0b763e-87bf-11f0-9565-1094bbd0ecec.jpg -> Annotation:/Users/briankimanzi/Documents/programmingLanguages/PythonProgramming/JupyterAI-Clean/Tensorflow/workspace/images/collectedimages/thankyou/thankyou.3e0b763e-87bf-11f0-9565-1094bbd0ecec.xml
Image:/Users/briankimanzi/Documents/programmingLanguages/PythonProgramming/JupyterAI-Clean/Tensorflow/workspace/images/collectedimages/thankyou/thankyou.70f4cf78-88be-11f0-8b4f-1094bbd0ecec.j

# Data training and testing

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [3]:
# BASE_PATH = os.path.expanduser("~/Documents/programmingLanguages/PythonProgramming/ObjectDetection/Tensorflow")  # goes to your Documents folder
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [5]:
files["PIPELINE_CONFIG"]

'Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config'

In [6]:
# This code makes the directories visible
for path in paths.values():
    os.makedirs(path, exist_ok=True)

In [7]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == "posix":
            !mkdir -p {path}
        if os.name == "nt":
            !mkdir -p {path}

<h2>Download TF Models Pretrained Models from tensorflow Model Zoo and Install TFOD</h2>

In [7]:
if os.name == "nt":
    !pip install wget
    import wget

In [4]:
# if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
#     !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}
!git clone https://github.com/tensorflow/models.git
!cd models/research

Cloning into 'models'...
remote: Enumerating objects: 102570, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 102570 (delta 58), reused 38 (delta 36), pack-reused 102482 (from 3)
Receiving objects: 100% (102570/102570), 642.78 MiB | 1.07 MiB/s, done.
Resolving deltas: 100% (74073/74073), done.
Updating files: 100% (3947/3947), done.


In [9]:
# # Install Tensorflow Object Detection 
# if os.name=='posix':  
#     !apt-get install protobuf-compiler
#     !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
# if os.name=='nt':
#     url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
#     wget.download(url)
#     !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
#     !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
#     os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
#     !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
#     !cd Tensorflow/models/research/slim && pip install -e . 

!pip install tensorflow==2.11.0 protobuf==3.20.3 numpy==1.23.5
!pip install tf-slim lxml pycocotools matplotlib

  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install protobuf==3.20.3 and tensorflow==2.11.0 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested protobuf==3.20.3
    tensorflow 2.11.0 depends on protobuf<3.20 and >=3.9.2

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [10]:
# Inside models/research/
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

# VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'],'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# # verify installation
# !python {VERIFICATION_SCRIPT}

zsh:1: no matches found: object_detection/protos/*.proto
cp: object_detection/packages/tf2/setup.py: No such file or directory
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [11]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!tar -xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
# !pip uninstall protobuf object-detection -y

zsh:1: command not found: wget
tar: Error opening archive: Failed to open 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'


In [8]:
!pip install tensorflow==2.11.0 protobuf==3.19.6 numpy==1.23.5 --force-reinstall

  Using cached tensorflow-2.11.0-cp310-cp310-macosx_10_14_x86_64.whl.metadata (3.1 kB)
  Using cached protobuf-3.19.6-py2.py3-none-any.whl.metadata (828 bytes)
  Using cached numpy-1.23.5-cp310-cp310-macosx_10_9_x86_64.whl.metadata (2.3 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.9.23-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.14.0-cp310-cp310-macosx_10_9_x86_64.whl.metadata (2.7 kB)
  Using cached libclang-18.1.1-py2.py3-none-macosx_10_9_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.me

In [8]:
pip list | grep protobuf

protobuf                  3.20.3
Note: you may need to restart the kernel to use updated packages.


In [19]:
!pip install tensorflow ==2.11.0

zsh:1: =2.11.0 not found


In [14]:
import object_detection

ModuleNotFoundError: No module named 'object_detection'

In [15]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

2025-09-30 04:24:06.639126: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available: 0
